In [1]:
import gym
import numpy as np
import torch
from agent import Agent
import matplotlib.pyplot as plt

In [2]:
# action adapter idea taken from:
# https://github.com/XinJingHao/PPO-Continuous-Pytorch
def action_adapter(a, max_a):
    return 2 * (a-0.5)*max_a



if __name__ == '__main__':
    env_id = 'Pendulum-v0'
    random_seed = 0
    torch.manual_seed(random_seed)
    np.random.seed(random_seed)
    env = gym.make(env_id)
    eval_env = gym.make(env_id)
    env.seed(random_seed)
    eval_env.seed(random_seed)
    N = 2048
    batch_size = 64
    n_epochs = 10
    alpha = 0.0003
    max_action = env.action_space.high[0]
    agent = Agent(n_actions=env.action_space.shape[0], batch_size=batch_size,
                  alpha=alpha, n_epochs=n_epochs,
                  input_dims=env.observation_space.shape)

    score_history = []
    max_steps = 1000000
    total_steps = 0
    traj_length = 0
    episode = 1

    # for i in range(n_games):
    while total_steps < max_steps:
        observation = env.reset()
        done = False
        score = 0
        while not done:
            action, prob = agent.choose_action(observation)
            act = action_adapter(action, max_action)
            observation_, reward, done, info = env.step(act)
            total_steps += 1
            traj_length += 1
            score += reward
            agent.remember(observation, observation_, action, prob, reward, done)
            if traj_length % N == 0:
                agent.learn()
                traj_length = 0
            observation = observation_
        score_history.append(score)
        avg_score = np.mean(score_history[-100:])
        print('{} Episode {} total steps {} avg score {:.1f}'.
              format(env_id, episode, total_steps, avg_score))
        episode += 1

c:\Users\Lenovo\Desktop\Python\Reinforcement Learning\5. PPO\Continuous\agent.py:30: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:233.)
  state = T.tensor([observation], dtype=T.float)


Pendulum-v0 Episode 1 total steps 200 avg score -1607.7
Pendulum-v0 Episode 2 total steps 400 avg score -1307.6
Pendulum-v0 Episode 3 total steps 600 avg score -1316.1
Pendulum-v0 Episode 4 total steps 800 avg score -1210.2
Pendulum-v0 Episode 5 total steps 1000 avg score -1288.0
Pendulum-v0 Episode 6 total steps 1200 avg score -1341.2
Pendulum-v0 Episode 7 total steps 1400 avg score -1280.6
Pendulum-v0 Episode 8 total steps 1600 avg score -1278.4
Pendulum-v0 Episode 9 total steps 1800 avg score -1235.0
Pendulum-v0 Episode 10 total steps 2000 avg score -1278.7
Pendulum-v0 Episode 11 total steps 2200 avg score -1248.2
Pendulum-v0 Episode 12 total steps 2400 avg score -1219.7
Pendulum-v0 Episode 13 total steps 2600 avg score -1214.0
Pendulum-v0 Episode 14 total steps 2800 avg score -1203.6
Pendulum-v0 Episode 15 total steps 3000 avg score -1184.4
Pendulum-v0 Episode 16 total steps 3200 avg score -1179.7
Pendulum-v0 Episode 17 total steps 3400 avg score -1163.7
Pendulum-v0 Episode 18 tota

In [ ]:
def plot_learning_curve(x, scores):
    running_avg = np.zeros(len(scores))
    for i in range(len(running_avg)):
        running_avg[i] = np.mean(scores[max(0, i-100):(i+1)])
    plt.plot(x, running_avg)
    plt.title('Running average of previous 100 scores')

x = [i+1 for i in range(len(score_history))]
plot_learning_curve(x, score_history)

: 

: 